# Change time column
Tableau interprets the time as a float. In this notebook we make it into a datetime object.

In [16]:
# Import necessary libraries
import os

import pandas as pd

In [87]:
# Load csv into dataframe
filename = os.path.join("..", "data", "clean_birdstrike_data.csv")
df = pd.read_csv(filename, low_memory=False, index_col=0)
df.head()

,Airport: Name,Altitude bin,Aircraft: Make/Model,Effect: Impact to flight,FlightDate,Record ID,Effect: Indicated Damage,Aircraft: Number of engines?,Aircraft: Airline/Operator,Origin State,...,Pilot warned of birds or wildlife?,Cost: Aircraft time out of service (hours),Cost: Other (inflation adj),Cost: Repair (inflation adj),Cost: Total $,Miles from airport,Feet above ground,Speed (IAS) in knots,lat,long
0,DETROIT METRO WAYNE COUNTY ARPT,< 1000 ft,A-319,NaN,1/1/2005 0:00,227413,No damage,2,NORTHWEST AIRLINES,Michigan,...,NaN,NaN,0,0,0,NaN,400,NaN,42.216172,-83.355384
1,DANE COUNTY REGIONAL ARPT-TRUAX FLD,< 1000 ft,A-320,NaN,1/1/2005 0:00,226578,No damage,2,NORTHWEST AIRLINES,Wisconsin,...,NaN,NaN,0,0,0,0,0,NaN,43.139066,-89.336413
2,HOGUE ARPT,< 1000 ft,PA-46 MALIBU,NaN,1/1/2005 0:00,268383,Caused damage,1,PRIVATELY OWNED,Tennessee,...,NaN,NaN,0,0,0,0,0,20,35.633724,-85.389510
3,SACRAMENTO INTL,< 1000 ft,B-737-300,None,1/1/2005 0:00,229514,Caused damage,2,SOUTHWEST AIRLINES,California,...,Y,NaN,0,"117,135","117,135",NaN,500,142,38.695085,-121.590065
4,GUARULHOS INTL,< 1000 ft,B-767-300,NaN,1/1/2005 0:00,225661,No damage,2,UNITED AIRLINES,NaN,...,NaN,NaN,0,0,0,0,0,NaN,-23.430573,-46.473043


In [3]:
# Check name of time column
df.columns

Index(['Unnamed: 0', 'Airport: Name', 'Altitude bin', 'Aircraft: Make/Model',
       'Effect: Impact to flight', 'FlightDate', 'Record ID',
       'Effect: Indicated Damage', 'Aircraft: Number of engines?',
       'Aircraft: Airline/Operator', 'Origin State', 'When: Phase of flight',
       'Wildlife: Size', 'Wildlife: Species', 'When: Time (HHMM)',
       'When: Time of day', 'Pilot warned of birds or wildlife?',
       'Cost: Aircraft time out of service (hours)',
       'Cost: Other (inflation adj)', 'Cost: Repair (inflation adj)',
       'Cost: Total $', 'Miles from airport', 'Feet above ground',
       'Speed (IAS) in knots', 'lat', 'long'],
      dtype='object')

In [58]:
# Check format of values
df[df["When: Time (HHMM)"].notna()]['When: Time (HHMM)'].sample(10)

46203     730.0
992      2300.0
63643    1620.0
19968    1810.0
6106     2229.0
45069     736.0
805      2335.0
25461    1143.0
5012     1019.0
21181    1000.0
Name: When: Time (HHMM), dtype: float64

In [88]:
# Transistion float to meaningful form

def time_change(x, mins):
    """Return x as HH:MM."""
    if pd.isna(x):
        return pd.NA
    time = str(x).split(".")[0].zfill(4)
    hour = time[:2]
    minute = time[2:]
    if int(minute) > 59:
        return pd.NA
    if mins:
        return f"1970-01-01 {hour}:{minute}"
    return f"1970-01-01 {hour}:00"

# First save only hours in new column, then full time in old column
df["Hours"] = df["When: Time (HHMM)"].apply(time_change, args=(False,))
df["When: Time (HHMM)"] = df["When: Time (HHMM)"].apply(time_change, args=(True,))

In [89]:
# Turn into datetime object
df["When: Time (HHMM)"] = pd.to_datetime(df["When: Time (HHMM)"])
df["Hours"] = pd.to_datetime(df["Hours"])

In [90]:
df[["Hours", "When: Time (HHMM)"]]

,Hours,When: Time (HHMM)
0,NaT,NaT
1,NaT,NaT
2,NaT,NaT
3,1970-01-01 21:00:00,1970-01-01 21:40:00
4,NaT,NaT
...,...,...
65605,1970-01-01 10:00:00,1970-01-01 10:25:00
65606,NaT,NaT
65607,1970-01-01 10:00:00,1970-01-01 10:26:00
65608,1970-01-01 12:00:00,1970-01-01 12:20:00


In [91]:
# Save to csv
df_filename = os.path.join("..", "data", "clean_birdstrike_data.csv")
df.to_csv(df_filename, index=False)